In [15]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/consumer/Edureka_Consumer_Complaints_train.csv
/kaggle/input/consumer/Edureka_Consumer_Complaints_test.csv


In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

test_df=pd.read_csv("/kaggle/input/consumer/Edureka_Consumer_Complaints_test.csv")
train_df=pd.read_csv("/kaggle/input/consumer/Edureka_Consumer_Complaints_train.csv")

# Function to analyze dataset

In [17]:
def dataset_details(dataframe):
    #print(dataframe.info(),"\n---------------------")
    print("Shape:",dataframe.shape,"\n---------------------")
    print("Duplicate count is",dataframe.duplicated().sum(),"\n---------------------")
    print("Missing count is",dataframe.isna().sum().sum(),"\n---------------------")
    missing_val_col=pd.DataFrame(dataframe.isna().sum())
    missing_val_col.rename(columns={0:"missing_val_count"},inplace=True)
    print("Out of",dataframe.shape[1],"columns, below are the columns having missing values")
    print(missing_val_col[~(missing_val_col["missing_val_count"]==0)],"\n---------------------")
    nan_rows_df=pd.DataFrame(dataframe.apply(lambda x: sum(x.isna()),axis=1))
    nan_rows_count=nan_rows_df[~(nan_rows_df[0]==0)].count().tolist()[0]
    print("Out of",dataframe.shape[0],"rows,",nan_rows_count,"have missing values","\n---------------------")
    print(dataframe.dtypes)


# Analyze Train dataset

In [18]:
print("-------------------------Analyzing Train dataset-------------------------")
dataset_details(train_df)

-------------------------Analyzing Train dataset-------------------------
Shape: (358810, 18) 
---------------------
Duplicate count is 0 
---------------------
Missing count is 1488297 
---------------------
Out of 18 columns, below are the columns having missing values
                              missing_val_count
Sub-product                              103786
Sub-issue                                219374
Consumer complaint narrative             302630
Company public response                  290879
State                                      2903
ZIP code                                   2911
Tags                                     308584
Consumer consent provided?               257230 
---------------------
Out of 358810 rows, 357640 have missing values 
---------------------
Date received                   object
Product                         object
Sub-product                     object
Issue                           object
Sub-issue                       object
Consumer

# Imputing missing value in column: Company public response 
* It has 2.9lakhs of missing entries out of 3.5lakh data

In [19]:
print(train_df["Company public response"].nunique())
print(train_df["Company public response"].value_counts())

10
Company chooses not to provide a public response                                                                           31526
Company has responded to the consumer and the CFPB and chooses not to provide a public response                            19268
Company believes it acted appropriately as authorized by contract or law                                                   12055
Company believes complaint caused principally by actions of third party outside the control or direction of the company     1091
Company believes the complaint is the result of a misunderstanding                                                          1075
Company disputes the facts presented in the complaint                                                                       1054
Company believes complaint is the result of an isolated error                                                                769
Company can't verify or dispute the facts in the complaint                                    

* From the generated output, we can infer that, the company has 10 public responses to choose from.
* For the entries, where Company pulic response is NaN, will assume "Company chooses not to provide a public response"

In [20]:
train_df["Company public response"]=train_df["Company public response"].replace(to_replace=np.nan,value="Company chooses not to provide a public response")
test_df["Company public response"]=test_df["Company public response"].replace(to_replace=np.nan,value="Company chooses not to provide a public response")

print("------Train-------\n")
print("Missing values in Company public response",train_df["Company public response"].isnull().sum())
print("------Test-------\n")
print("Missing values in Company public response",test_df["Company public response"].isnull().sum())

------Train-------

Missing values in Company public response 0
------Test-------

Missing values in Company public response 0


# Create new column: Days difference
* Days difference = Differnce between "Date sent to company" and "Date received" by converting into datetime 
* [ An anamoly is observed in the dataset, as for some entries Date sent to the company is lesser than the date at which complaint is recieved. Considering it as an data entry mistake, will take absolute values to correct it ]

In [21]:
###### New Column: Days difference on Train

train_df["Date received"]=pd.to_datetime(train_df["Date received"])
train_df["Date sent to company"]=pd.to_datetime(train_df["Date sent to company"])
train_df["Days difference"]=np.abs(train_df["Date sent to company"]-train_df["Date received"])
train_df["Days difference"]=train_df["Days difference"].apply(lambda x: str(x).strip()[0])

###### New Column: Days difference on Test

test_df["Date received"]=pd.to_datetime(test_df["Date received"])
test_df["Date sent to company"]=pd.to_datetime(test_df["Date sent to company"])
test_df["Days difference"]=np.abs(test_df["Date sent to company"]-test_df["Date received"])
test_df["Days difference"]=test_df["Days difference"].apply(lambda x: str(x).strip()[0])

In [22]:
train_df["Days difference"].value_counts()

0    157154
1     63194
2     35203
3     26420
4     23664
5     20603
6     15582
7     10331
8      4241
9      2418
Name: Days difference, dtype: int64

* On analyzing the output,we can infer:-
* 1. Days difference between the date at which the complaint is recieved and sent to the company, ranges between 0 to 9. 
* 2. Mostly the complaints are sent to the company on the same day it is recieved.


In [23]:
train_df.describe(include="O")

,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Company response to consumer,Timely response?,Consumer disputed?,Days difference
count,358810,255024,358810,139436,56180,358810,358810,355907,355899,50226,101580,358810,358810,358810,358810,358810
unique,12,47,95,67,55535,10,3064,62,24181,3,4,6,6,2,2,10
top,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",Account status,This company continues to report on my credit ...,Company chooses not to provide a public response,Bank of America,CA,300XX,Older American,Consent provided,Web,Closed with explanation,Yes,No,0
freq,117066,46174,60185,17142,26,322405,34903,52368,910,29253,56181,235462,265700,352675,282638,157154


# Create new dataframe

* To predict, if the Consumer, disputed with the solution provided then below are the columns to drop:-

*   1. ComplaintID - unique id given to each complain [ Irrelevant,as machine will learn on what kind of complaints are getting disputed ]
*   2. Consumer complaint narrative - Desciption of complain [Irrelevant, as we are considering the product column. On examinig "Consumer complaint narrative" we can get the category of the  product ]
*   3. Consumer consent provided? - whether customer gave consent on publishing the complain [ Irrelevant,as it is just the customer consent,for the complain publishment ]
*   4. Date received - Date on which complaint was registered [ Irrelevant,as machine will learn on what kind of complaints are getting disputed ]
*   5. Date sent to company - Date on which complaint was sent to the respective company [ Irrelevant,as machine will learn on what kind of complaints are getting disputed ]
*   6. Tags - [ Irrelevant,as 3lakhs entries were missing out of 3.5lakh data ]
*   7. Zip code - [ Irrelevant,as some are encoded with XX ]
*   8. Sub-product
*   9. Sub-issue
*   10. State

In [24]:
train_new_df=train_df.drop(["Date received","Sub-product","Sub-issue","State","Consumer complaint narrative","Consumer consent provided?","Date sent to company","Complaint ID","Tags","ZIP code"],axis=1)
test_new_df=test_df.drop(["Date received","Sub-product","Sub-issue","State","Consumer complaint narrative","Consumer consent provided?","Date sent to company","Complaint ID","Tags","ZIP code"],axis=1)

# Label Encoding

In [25]:
col=train_new_df.drop(["Consumer disputed?","Company","Issue"],axis=1).columns.tolist()

from sklearn.preprocessing import LabelEncoder
lr=LabelEncoder()

##Encoding variable
train_new_df["Consumer disputed?"]=lr.fit_transform(train_new_df["Consumer disputed?"])
train_new_df["Company"]=lr.fit_transform(train_new_df["Company"])
test_new_df["Company"]=lr.fit_transform(test_new_df["Company"])
train_new_df["Issue"]=lr.fit_transform(train_new_df["Issue"])
test_new_df["Issue"]=lr.fit_transform(test_new_df["Issue"])


dummy_train=pd.get_dummies(train_new_df,columns=col)
dummy_test=pd.get_dummies(test_new_df,columns=col)
dummy_train.head()
    


,Issue,Company,Consumer disputed?,Product_Bank account or service,Product_Consumer Loan,Product_Credit card,Product_Credit reporting,Product_Debt collection,Product_Money transfers,Product_Mortgage,...,Days difference_0,Days difference_1,Days difference_2,Days difference_3,Days difference_4,Days difference_5,Days difference_6,Days difference_7,Days difference_8,Days difference_9
0,52,998,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,39,2986,1,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,68,623,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,40,1031,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
4,40,1320,1,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0


In [26]:
dummy_test.head()

,Issue,Company,Product_Bank account or service,Product_Consumer Loan,Product_Credit card,Product_Credit reporting,Product_Debt collection,Product_Money transfers,Product_Mortgage,Product_Other financial service,...,Days difference_0,Days difference_1,Days difference_2,Days difference_3,Days difference_4,Days difference_5,Days difference_6,Days difference_7,Days difference_8,Days difference_9
0,35,457,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,72,1168,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,31,399,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,76,1533,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,49,910,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [27]:
XX=dummy_train.drop("Consumer disputed?",axis=1)
YY=dummy_train["Consumer disputed?"]

# Split the dataset

In [28]:
from sklearn.model_selection import train_test_split
x_train1,x_test1,y_train1,y_test1=train_test_split(XX,YY,test_size=0.3,random_state=101)

x_train1.head()

,Issue,Company,Product_Bank account or service,Product_Consumer Loan,Product_Credit card,Product_Credit reporting,Product_Debt collection,Product_Money transfers,Product_Mortgage,Product_Other financial service,...,Days difference_0,Days difference_1,Days difference_2,Days difference_3,Days difference_4,Days difference_5,Days difference_6,Days difference_7,Days difference_8,Days difference_9
309691,16,1024,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
65522,39,1970,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
299529,39,2986,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
67161,78,2550,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
232016,52,998,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


# Find which model gives better result

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
#from sklearn.svm import LinearSVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix,f1_score

models=[LogisticRegression(max_iter=2000),RandomForestClassifier(),MultinomialNB(),XGBClassifier()]
for i in models:
    i.fit(x_train1,y_train1)
    pred=i.predict(x_test1)
    print(accuracy_score(y_test1,pred))
    result=f1_score(y_test1,pred, average = "macro")
    print("f1_score of",i,"is : ",result)
    print(confusion_matrix(y_test1,pred))

0.7876313369192609
f1_score of LogisticRegression(max_iter=2000) is :  0.44060054254622555
[[84783     0]
 [22860     0]]
0.760811199985136
f1_score of RandomForestClassifier() is :  0.4895188239930269
[[80184  4599]
 [21148  1712]]
0.7876127569837332
f1_score of MultinomialNB() is :  0.44059472830831914
[[84781     2]
 [22860     0]]
0.7876870767258438
f1_score of XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None) is :  0.44523265925963335
[[84681   102]
 [22752  

# Cross Validation

In [30]:
##KFOLD for accuracy
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kfold=KFold(n_splits=3,random_state=101)
models=[LogisticRegression(max_iter=2000),RandomForestClassifier(),MultinomialNB(),XGBClassifier()]
for i in models:
    print("Accuracy of",i,"is : ",np.round(np.mean(cross_val_score(i,XX,YY,cv=kfold,scoring='accuracy')),3))

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Accuracy of LogisticRegression(max_iter=2000) is :  0.788
Accuracy of RandomForestClassifier() is :  0.759
Accuracy of MultinomialNB() is :  0.788
Accuracy of XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              validate_parameters=None, verbosity=None) is :  0.788


*** From the results, it is clear that XGBClassifier does a better job on the imbalance dataset**

# Find best parameter for different model

In [13]:
from xgboost import XGBClassifier
XGBClassifier().get_params().keys() 

dict_keys(['objective', 'base_score', 'booster', 'colsample_bylevel', 'colsample_bynode', 'colsample_bytree', 'gamma', 'gpu_id', 'importance_type', 'interaction_constraints', 'learning_rate', 'max_delta_step', 'max_depth', 'min_child_weight', 'missing', 'monotone_constraints', 'n_estimators', 'n_jobs', 'num_parallel_tree', 'random_state', 'reg_alpha', 'reg_lambda', 'scale_pos_weight', 'subsample', 'tree_method', 'validate_parameters', 'verbosity'])

In [14]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
grid={"max_depth": range(2,10,1),"n_estimators": range(60,220,40),"learning_rate" : [0.1, 0.01,0.05]}
random=RandomizedSearchCV(estimator=XGBClassifier(), param_distributions=grid, n_jobs=-1, cv=10, scoring='accuracy')
random.fit(x_train1,y_train1)
print(random.best_score_,random.best_params_)
print(random.best_estimator_)

0.7878463332137529 {'n_estimators': 140, 'max_depth': 5, 'learning_rate': 0.1}
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=140, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)


*** Tunning XGBClassifier hyperparameters didn't make much of an impact on accuracy.**

# Prediction on test dataset

In [31]:
XXX=dummy_train.drop("Consumer disputed?",axis=1)
YYY=dummy_train["Consumer disputed?"]

from xgboost import XGBClassifier
xgb=XGBClassifier()
xgb.fit(XXX,YYY)
predictions=xgb.predict(dummy_test)
resul=pd.Series(predictions)
resul.to_csv("/kaggle/working/model_prediction.csv",index=False)
print("done")

done
